## Load in R Packages

In [ ]:
library("Eunomia")
library("DatabaseConnector")
library("CohortMethod")
library(PatientLevelPrediction)
library("ResultModelManager")
library(ShinyAppBuilder)

## Connect to Eunomia GiBleed dataset (default) and Test a Query

In [ ]:
# Got an error once and had to do this to resolve...
#options(databaseConnectorInteger64AsNumeric = FALSE)


connectionDetails <- getEunomiaConnectionDetails()


createCohorts(connectionDetails)
connection <- DatabaseConnector::connect(connectionDetails)

# This is how you would connect to a different dataset
#conn <- DatabaseConnector::connect(dbms = "sqlite", server = getDatabaseFile("MIMIC"))

# Get top ppl (use limit in sqlite, not top)
DatabaseConnector::querySql(connection, sql="SELECT * FROM person;")

# Another example query...
#DatabaseConnector::getTableNames(connection,databaseSchema = 'main')



## Do Cohort Method Exercise from Book of Ohdsi
[Exercise 12.1](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple1)

In [ ]:
nsaids <- c(1118084, 1124300) # celecoxib, diclofenac
covSettings <- createDefaultCovariateSettings(
  excludedCovariateConceptIds = nsaids,
  addDescendantsToExclude = TRUE)

# Load data:
cmData <- getDbCohortMethodData(
  connectionDetails = connectionDetails,
  cdmDatabaseSchema = "main",
  targetId = 1,
  comparatorId = 2,
  outcomeIds = 3,
  exposureDatabaseSchema = "main",
  exposureTable = "cohort",
  outcomeDatabaseSchema = "main",
  outcomeTable = "cohort",
  covariateSettings = covSettings)
summary(cmData)

## Create Study Population 
Book of OHDSI [Exercise 12.2](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple2)

In [ ]:
studyPop <- createStudyPopulation(
  cohortMethodData = cmData,
  outcomeId = 3,
  washoutPeriod = 180,
  removeDuplicateSubjects = "remove all",
  removeSubjectsWithPriorOutcome = TRUE,
  riskWindowStart = 0,
  startAnchor = "cohort start",
  riskWindowEnd = 99999)
drawAttritionDiagram(studyPop)

## Cox Regression
Book of OHDSI [Exercise 12.3](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple3)

In [ ]:
model <- fitOutcomeModel(population = studyPop,
                         modelType = "cox")
model

## Propensity Model
Book of OHDSI [Exercise 12.4](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple4)

In [ ]:
ps <- createPs(cohortMethodData = cmData,
               population = studyPop)
plotPs(ps, showCountsLabel = TRUE, showAucLabel = TRUE)

## PS Stratification using 5 Strata
Book of OHDSI [Exercise 12.5](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple5)

In [ ]:
strataPop <- stratifyByPs(ps, numberOfStrata = 5)
bal <- computeCovariateBalance(strataPop, cmData)
plotCovariateBalanceScatterPlot(bal,
                                showCovariateCountLabel = TRUE,
                                showMaxLabel = TRUE,
                                beforeLabel = "Before stratification",
                                afterLabel = "After stratification")

## Outcome Model using Cox Regression
Book of OHDSI [Exercise 12.6](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseple6)

In [ ]:
adjModel <- fitOutcomeModel(population = strataPop,
                         modelType = "cox",
                         stratified = TRUE)
adjModel

## Define Covariates for Prediction and Extract PLP data from CDM
Book of OHDSI [Exercise 13.1](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseplp1)
Note: This is updated from the book, since it seems the book is out of date...

In [ ]:
dbDetails <- createDatabaseDetails(
    connectionDetails = connectionDetails,
    cdmDatabaseSchema = "main",
    cdmDatabaseName = "main",
    cdmDatabaseId = "main",
    outcomeTable = "cohort",
    outcomeIds = 3,
    cohortId = 4
)
covSettings <- createCovariateSettings(
  useDemographicsGender = TRUE,
  useDemographicsAge = TRUE,
  useConditionGroupEraLongTerm = TRUE,
  useConditionGroupEraAnyTimePrior = TRUE,
  useDrugGroupEraLongTerm = TRUE,
  useDrugGroupEraAnyTimePrior = TRUE,
  useVisitConceptCountLongTerm = TRUE,
  longTermStartDays = -365,
  endDays = -1)

plpData <- getPlpData(
    databaseDetails = dbDetails,
    covariateSettings = covSettings,
    restrictPlpDataSettings = createRestrictPlpDataSettings()
)

summary(plpData)

## Create Study Population
Book of OHDSI [Exercise 13.2](https://ohdsi.github.io/TheBookOfOhdsi/SuggestedAnswers.html#exercise-refexrexerciseplp2)

In [ ]:
populationSettings <- createStudyPopulationSettings(
    washoutPeriod = 364,
    firstExposureOnly = F,
    removeSubjectsWithPriorOutcome = T,
    priorOutcomeLookback = 9999,
    riskWindowStart = 1,
    riskWindowEnd = 365,
    #addExposureDaysToStart = F,
    #addExposureDaysToEnd = F,
    startAnchor = "cohort start",
    endAnchor = "cohort start",
    minTimeAtRisk = 364,
    requireTimeAtRisk = T,
    includeAllOutcomes = T

)
population <- createStudyPopulation(
    plpData = plpData, 
    populationSettings = populationSettings,
    outcomeId = 3
)
nrow(population)

13.3

In [ ]:
lassoModel <- setLassoLogisticRegression()
#plpData$population = population
lassoResults <- runPlp(
                        #population = population,
                        populationSettings = populationSettings,
                       plpData = plpData,
                       modelSettings = lassoModel,
                       outcomeId = 3,
                       #testSplit = 'person',
                       #testFraction = 0.25,
                       splitSettings = createDefaultSplitSetting(type = "stratified", testFraction = 0.25,trainFraction = 0.75, splitSeed = 0, nfold = 3)
                       #nfold = 2,
                       #splitSeed = 0
                       )

In [ ]:
library(ShinyAppBuilder)
viewPlp(lassoResults)

15.1

15.2

15.3

## Disconnect

In [ ]:

DatabaseConnector::disconnect(connection)